In [6]:
import glob, os
import pandas

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from struct import *

<h3>CLASSIFICATION CODE</h3>
This part is the same as in IoT-Sentinel study except for the part where they use Damerau-Levenshtein edit distance 

In [78]:
def classify(device_name, device_route):
    #print(device_route)
    i = 0
    df_list = []
    headers_name=['remove','ARP','LLC','EAPOL','Pck_size','Pck_rawdata','IP_padding','IP_ralert','IP_add_count','Portcl_src','Portcl_dst','ICMP','ICMP6','TCP','UDP','HTTPS','HTTP','DHCP','BOOTP','SSDP','DNS','MDNS','NTP','Label']
    
    #Extract panda DataFrame from .csv
    for filename in os.listdir(device_route):
       
        filepath= os.path.join(device_route, filename)
        
        df = pandas.read_csv(filepath ,sep='\t', encoding='utf-8', header=None)
        df.columns=headers_name
        df= df.drop('remove', axis=1)
        df_list.append(df)
        i +1
        
    
    data = pandas.concat(df_list)
    
    #Append data from other devices
    globals()["Data_Mixed"]= Data_Mixed.reset_index(drop=True)
    globals()["Data_Mixed_Without"] = Data_Mixed.drop(Data_Mixed.loc[Data_Mixed['Label'] == device_name, Data_Mixed.columns].index)
   
    #Reduce data
    if(data.shape[0]>20000):
        print("Data is reduced to 20000")
        data= data.sample(n=20000)
        
    #BALANCE DATA
    # Obtener el número de filas de cada dataframe
    num_rows_df1 = Data_Mixed_Without.shape[0]
    num_rows_df2 = data.shape[0]
    #print(num_rows_df1)
    #print(num_rows_df2)
    
    # Seleccionar el número de filas para cada dataframe
    num_rows = min(num_rows_df1, num_rows_df2)
    print("Num rows is: ")
    print(num_rows)
    
    # Submuestrear cada dataframe para tener el mismo número de filas
    df1_sample = Data_Mixed_Without.sample(n=num_rows)
    df2_sample = data.sample(n=num_rows)

    # Combinar los dataframes submuestreados
    data = pandas.concat([df1_sample, df2_sample])
    
    
    #Train the classifiers
    trainD = data.drop('Label', axis=1)
    testD = data['Label']
    X_train, X_test, y_train, y_test = train_test_split(trainD, testD, test_size=0.2, random_state=42)    

    # Initialize a random forest classifier with default parameters
    clf = RandomForestClassifier(n_estimators=100, max_depth=5)
    
    # Train the random forest classifier on the training set
    clf.fit(X_train, y_train)
    
    X_test['Label']= y_test 
    X_test= X_test.drop(X_test[X_test['Label'] == "other"].index, axis=0)
    y_test= X_test['Label']
    X_test= X_test.drop('Label', axis=1) 
    
    return (clf, X_test,y_test)

In [79]:
def chooseNearest(predictions,row):
    highProb = 0
    for prediction in predictions:
        device = prediction.replace("-","_")
        clf = device.replace("-","_") +"_Class"
        prob = globals()[clf].predict_proba(row)[:, 0]
        if prob > highProb:
            highProb= prob
            resul= prediction
    return resul

In [80]:
def test():
    
    
    headers_name=['ARP','LLC','EAPOL','Pck_size','Pck_rawdata','IP_padding','IP_ralert','IP_add_count','Portcl_src','Portcl_dst','ICMP','ICMP6','TCP','UDP','HTTPS','HTTP','DHCP','BOOTP','SSDP','DNS','MDNS','NTP','Label']
    
    for dataCounter in range(31):
        match = 0
        clfCounter = 0
        var_name= Devices[dataCounter].replace("-","_") +"_Data"
        data= globals()[var_name]
        
        predictions_all= []
        
        test_label= Devices[dataCounter].replace("-","_") +"_Data_Label"
        correctResul= globals()[test_label]
        
        print("\nStarting testing: "+ Devices[dataCounter])
        for i in range(len(data)):
            X= data.iloc[i]
            X= X.reset_index(drop=True)
            row= X.values.reshape(1,-1)
            row= pandas.DataFrame(row,columns=headers_name[:-1])
            
            predictions = []
            
            if Aria_Class.predict(row)=='Aria':
                predictions.append('Aria')
            if D_LinkCam_Class.predict(row)=='D-LinkCam':
                predictions.append('D-LinkCam')
            if D_LinkDayCam_Class.predict(row)=='D-LinkDayCam':
                predictions.append('D-LinkDayCam')
            if D_LinkDoorSensor_Class.predict(row)=='D-LinkDoorSensor':
                predictions.append('D-LinkDoorSensor')
            if D_LinkHomeHub_Class.predict(row)=='D-LinkHomeHub':
                predictions.append('D-LinkHomeHub')    
            if D_LinkHomeHub_Class.predict(row) == 'D-LinkHomeHub':
                predictions.append('D-LinkHomeHub')
            if D_LinkSensor_Class.predict(row) == 'D-LinkSensor':
                predictions.append('D-LinkSensor')
            if D_LinkSiren_Class.predict(row) == 'D-LinkSiren':
                predictions.append('D-LinkSiren')
            if D_LinkSwitch_Class.predict(row) == 'D-LinkSwitch':
                predictions.append('D-LinkSwitch')
            if D_LinkWaterSensor_Class.predict(row) == 'D-LinkWaterSensor':
                predictions.append('D-LinkWaterSensor')
            if EdimaxCam1_Class.predict(row) == 'EdimaxCam1':
                predictions.append('EdimaxCam1')
            if EdimaxCam2_Class.predict(row) == 'EdimaxCam2':
                predictions.append('EdimaxCam2')
            if EdimaxPlug1101W_Class.predict(row)=='EdimaxPlug1101W':
                predictions.append('EdimaxPlug1101W')
            if EdimaxPlug2101W_Class.predict(row)=='EdimaxPlug2101W':
                predictions.append('EdimaxPlug2101W')
            if EdnetCam1_Class.predict(row)=='EdnetCam1':
                predictions.append('EdnetCam1')
            if EdnetCam2_Class.predict(row)=='EdnetCam2':
                predictions.append('EdnetCam2')
            if EdnetGateway_Class.predict(row)=='EdnetGateway':
                predictions.append('EdnetGateway')
            if HomeMaticPlug_Class.predict(row)=='HomeMaticPlug':
                predictions.append('HomeMaticPlug')
            if HueBridge_Class.predict(row)=='HueBridge':
                predictions.append('HueBridge')
            if HueSwitch_Class.predict(row)=='HueSwitch':
                predictions.append('HueSwitch')
            if iKettle2_Class.predict(row)=='iKettle2':
                predictions.append('iKettle2')
            if Lightify_Class.predict(row)=='Lightify':
                predictions.append('Lightify')
            if MAXGateway_Class.predict(row)=='MAXGateway':
                predictions.append('MAXGateway')
            if SmarterCoffee_Class.predict(row)=='SmarterCoffee':
                predictions.append('SmarterCoffee')
            if TP_LinkPlugHS100_Class.predict(row)=='TP-LinkPlugHS100':
                predictions.append('TP-LinkPlugHS100')
            if TP_LinkPlugHS110_Class.predict(row)=='TP-LinkPlugHS110':
                predictions.append('TP-LinkPlugHS110')
            if WeMoInsightSwitch_Class.predict(row)=='WeMoInsightSwitch':
                predictions.append('WeMoInsightSwitch')
            if WeMoInsightSwitch2_Class.predict(row)=='WeMoInsightSwitch2':
                predictions.append('WeMoInsightSwitch2')
            if WeMoLink_Class.predict(row)=='WeMoLink':
                predictions.append('WeMoLink')
            if WeMoSwitch_Class.predict(row)=='WeMoSwitch':
                predictions.append('WeMoSwitch')
            if WeMoSwitch2_Class.predict(row)=='WeMoSwitch2':
                predictions.append('WeMoSwitch2')
            if Withings_Class.predict(row)=='Withings':
                predictions.append('Withings')
                
            predictions_all.append(chooseNearest(predictions,row))
        accuracy= accuracy_score(correctResul,predictions_all)
        print("Acurracy for "+ Devices[dataCounter] + " is: "  )
        print(f"{accuracy}\n")
       
        dataCounter +1
            
        
        

In [81]:
def takedata():
    input_dir = "csv_result_full/"
    df_list=[]
    headers_name=['remove','ARP','LLC','EAPOL','Pck_size','Pck_rawdata','IP_padding','IP_ralert','IP_add_count','Portcl_src','Portcl_dst','ICMP','ICMP6','TCP','UDP','HTTPS','HTTP','DHCP','BOOTP','SSDP','DNS','MDNS','NTP','Label']
    
    for name in os.listdir(input_dir):
        if os.path.isdir(os.path.join(input_dir, name)): 
            device_route= input_dir + name
            for filename in os.listdir(device_route):
                filepath= os.path.join(device_route, filename)
                df = pandas.read_csv(filepath ,sep='\t', encoding='utf-8', header=None)
                df.columns= headers_name
                df= df.drop('remove', axis=1)
                df_list.append(df)
    data = pandas.concat(df_list)
    globals()["Data_Mixed"]= data            

In [82]:


def main():
    #TEST DATA
    global Aria_Data
    global D_LinkCam_Data
    global D_LinkDayCam_Data
    global D_LinkDoorSensor_Data
    global D_LinkHomeHub_Data
    global D_LinkSensor_Data
    global D_LinkSiren_Data
    global D_LinkSwitch_Data
    global D_LinkWaterSensor_Data
    global EdimaxCam1_Data
    global EdimaxCam2_Data
    global EdimaxPlug1101W_Data
    global EdimaxPlug2101W_Data
    global EdnetCam1_Data
    global EdnetCam2_Data
    global EdnetGateway_Data
    global HomeMaticPlug_Data
    global HueBridge_Data
    global HueSwitch_Data
    global iKettle2_Data
    global Lightify_Data
    global MAXGateway_Data
    global SmarterCoffee_Data
    global TP_LinkPlugHS100_Data
    global TP_LinkPlugHS110_Data
    global WeMoInsightSwitch_Data
    global WeMoInsightSwitch2_Data
    global WeMoLink_Data
    global WeMoSwitch_Data
    global WeMoSwitch2_Data
    global Withings_Data

    #TEST CORRECT LABELS
    global Aria_Data_Label
    global D_LinkCam_Data_Label
    global D_LinkDayCam_Data_Label
    global D_LinkDoorSensor_Data_Label
    global D_LinkHomeHub_Data_Label
    global D_LinkSensor_Data_Label
    global D_LinkSiren_Data_Label
    global D_LinkSwitch_Data_Label
    global D_LinkWaterSensor_Data_Label
    global EdimaxCam1_Data_Label
    global EdimaxCam2_Data_Label
    global EdimaxPlug1101W_Data_Label
    global EdimaxPlug2101W_Data_Label
    global EdnetCam1_Data_Label
    global EdnetCam2_Data_Label
    global EdnetGateway_Data_Label
    global HomeMaticPlug_Data_Label
    global HueBridge_Data_Label
    global HueSwitch_Data_Label
    global iKettle2_Data_Label
    global Lightify_Data_Label
    global MAXGateway_Data_Label
    global SmarterCoffee_Data_Label
    global TP_LinkPlugHS100_Data_Label
    global TP_LinkPlugHS110_Data_Label
    global WeMoInsightSwitch_Data_Label
    global WeMoInsightSwitch2_Data_Label
    global WeMoLink_Data_Label
    global WeMoSwitch_Data_Label
    global WeMoSwitch2_Data_Label
    global Withings_Data_Label

    #CLASSIFIYERS
    global Aria_Class
    global D_LinkCam_Class
    global D_LinkDayCam_Class
    global D_LinkDoorSensor_Class
    global D_LinkHomeHub_Class
    global D_LinkSensor_Class
    global D_LinkSiren_Class
    global D_LinkSwitch_Class
    global D_LinkWaterSensor_Class
    global EdimaxCam1_Class
    global EdimaxCam2_Class
    global EdimaxPlug1101W_Class
    global EdimaxPlug2101W_Class
    global EdnetCam1_Class
    global EdnetCam2_Class
    global EdnetGateway_Class
    global HomeMaticPlug_Class
    global HueBridge_Class
    global HueSwitch_Class
    global iKettle2_Class
    global Lightify_Class
    global MAXGateway_Class
    global SmarterCoffee_Class
    global TP_LinkPlugHS100_Class
    global TP_LinkPlugHS110_Class
    global WeMoInsightSwitch_Class
    global WeMoInsightSwitch2_Class
    global WeMoLink_Class
    global WeMoSwitch_Class
    global WeMoSwitch2_Class
    global Withings_Class

    global Data_Mixed
    global Data_Mixed_Without

    global Devices  
   
    Devices = ['Aria', 'D-LinkCam', 'D-LinkDayCam', 'D-LinkDoorSensor', 'D-LinkHomeHub', 'D-LinkSensor', 'D-LinkSiren', 'D-LinkSwitch', 'D-LinkWaterSensor', 'EdimaxCam1', 'EdimaxCam2', 'EdimaxPlug1101W', 'EdimaxPlug2101W', 'EdnetCam1', 'EdnetCam2', 'EdnetGateway', 'HomeMaticPlug', 'HueBridge', 'HueSwitch', 'iKettle2', 'Lightify', 'MAXGateway', 'SmarterCoffee', 'TP-LinkPlugHS100', 'TP-LinkPlugHS110', 'WeMoInsightSwitch', 'WeMoInsightSwitch2', 'WeMoLink', 'WeMoSwitch', 'WeMoSwitch2', 'Withings']

    input_dir = "csv_result_full/"
    file_pattern ='*.csv'
    
    
    takedata() 
    for name in os.listdir(input_dir):
        i=0
        test_label=[]
        devices=[]
        if os.path.isdir(os.path.join(input_dir, name)):
            device_name= name
            devices.append(name)
            print ('\nDEVICE CLASSIFYING: '+device_name + '\n')
            
            device_route = input_dir + name
            dmw= Data_Mixed
            #print(Data_Mixed)
            Data_Mixed_Without= dmw.drop(dmw.loc[dmw['Label'] == name].index)
            #print(Data_Mixed_Without.shape[0])
            Data_Mixed_Without['Label']= "other"
            
            clfi ,test_datai,test_labeli = classify(name,device_route)
            
            
            var_name= device_name.replace("-","_") +"_Class"
            globals()[var_name]=clfi
            
            var_name= device_name.replace("-","_") +"_Data"
            globals()[var_name]= test_datai
            
            var_name= device_name.replace("-","_") +"_Data_Label"
            globals()[var_name]= test_labeli
            
            i +1
       
   
    #now test
    """
    pred= HueSwitch_Class.predict(Aria_Data)
    print(pred)
    prob = HueSwitch_Class.predict_proba(Aria_Data)[:, 0]
    print(prob)
    """
    print('\nCLASSIFICATION COMPLETED \n')
    print ('\nSTARTING TESTING...\n')
    test( )
    

main()


DEVICE CLASSIFYING: Aria

Data_Mixed:
391722
Data:
6734
Data_Mixed_Without:
384988

DEVICE CLASSIFYING: D-LinkCam

Data_Mixed:
391722
Data:
15018
Data_Mixed_Without:
376704

DEVICE CLASSIFYING: D-LinkDayCam

Data_Mixed:
391722
Data:
2470
Data_Mixed_Without:
389252

DEVICE CLASSIFYING: D-LinkDoorSensor

Data_Mixed:
391722
Data:
7592
Data_Mixed_Without:
384130

DEVICE CLASSIFYING: D-LinkHomeHub

Data_Mixed:
391722
Data:
31756
Data_Mixed_Without:
359966
Data is reduced to 20000

DEVICE CLASSIFYING: D-LinkSensor

Data_Mixed:
391722
Data:
25382
Data_Mixed_Without:
366340
Data is reduced to 20000

DEVICE CLASSIFYING: D-LinkSiren

Data_Mixed:
391722
Data:
23626
Data_Mixed_Without:
368096
Data is reduced to 20000

DEVICE CLASSIFYING: D-LinkSwitch

Data_Mixed:
391722
Data:
25900
Data_Mixed_Without:
365822
Data is reduced to 20000

DEVICE CLASSIFYING: D-LinkWaterSensor

Data_Mixed:
391722
Data:
24196
Data_Mixed_Without:
367526
Data is reduced to 20000

DEVICE CLASSIFYING: EdimaxCam1

Data_Mixed

KeyboardInterrupt: 